In [ ]:
import pandas as pd
from pandas import DataFrame, Series
import numpy as np
from IPython.display import display, HTML
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
%matplotlib inline
import nmrglue as ng
import glob

In [ ]:
filenames = glob.glob("*MQF*.fid")
experiment_list = []
for filename in filenames:
    dic, FIDs = ng.varian.read(filename)
    count = 0
    for i in range(np.shape(FIDs)[0]):
        thisFID = FIDs[i]
        if np.sum(thisFID) == 0.0 or np.max(np.abs(thisFID)) < 0.1:
            print "Ignored blank FID %d from %s." % (i+1, filename)
            continue
        count += 1
        experiment_list.append(FIDs[i])    
    npoints = np.shape(experiment_list[0])[0]
    print "Sequence: %s (%s=%s, %s=%s)" % (dic["procpar"]["seqfil"]["values"][0], dic["procpar"]["dn"]["values"][0],
                                           dic["procpar"]["dm"]["values"][0], dic["procpar"]["dn2"]["values"][0],
                                           dic["procpar"]["dm2"]["values"][0])
    print "%d FIDs loaded from %s (%d complex points, nt=%sx%s, d1=%s s)." % (count, filename, npoints, len(dic["procpar"]["nt"]["values"]), dic["procpar"]["nt"]["values"][0], dic["procpar"]["d1"]["values"][0])

In [ ]:
obs = float(dic["procpar"]["reffrq"]["values"][0])
sw  = float(dic["procpar"]["sw"]["values"][0])  
tof = float(dic["procpar"]["tof"]["values"][0])     
carrier = obs*1.0E6 + tof                           # carrier frequency in Hz

udic = ng.varian.guess_udic(dic, FIDs)  
udic[0]['size']     = int(dic["np"])   # number of R|I points in the spectrum
udic[0]['complex']  = True             # True if complex data
udic[0]['encoding'] = 'direct'         # keep as 'direct'
udic[0]['sw']       = sw               # spectral width in Hz
udic[0]['obs']      = obs              # Observation freq. in MHz.
udic[0]['car']      = carrier          # carrier freq in Hz
udic[0]['label']    = 'F19'            # the observed nucleus
udic[0]['time']     = True             # whether this is time domain data
udic[0]['freq']     = False
udic["ndim"]        = 1

In [ ]:
def process(dic, FID):
    C = ng.convert.converter()
    C.from_varian(dic, FID, udic)
    pdic, pdata = C.to_pipe()
    pdic, pdata = ng.pipe_proc.em(pdic, pdata, lb=0.25)                 # line broadening
    pdic, pdata = ng.pipe_proc.zf(pdic, pdata, size=4*npoints, auto=True)
    pdic, pdata = ng.pipe_proc.ft(pdic, pdata)
    pdic, pdata = ng.pipe_proc.ps(pdic, pdata, p0=202, p1=0)            # phase correction
    return pdic, pdata

In [ ]:
raw_spectrum_list = [ process(dic,FID) for FID in experiment_list ]
raw_spectra = [ spectrum for dic, spectrum in raw_spectrum_list ]
    
pdic, pdata = raw_spectrum_list[1]
uc = ng.pipe.make_uc(pdic, pdata)
ppm = uc.ppm_scale()
ppm = ppm - ppm[-1] - 239.525        
  
spectrum_number = 0       
spectrum = raw_spectra[spectrum_number]
plt.figure(figsize=(18,4))
plt.plot(ppm, np.real(spectrum), "k")
plt.xlim(-50,-230)      
plt.ylim(-2E6,1E7)      

plt.xlabel("19F chemical shift (ppm)")
ax = plt.gca()
ax.tick_params(top="off")
ax.get_xaxis().set_tick_params(length=5,direction='out', width=1)
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)
plt.show()

In [ ]:
interval = ppm[1]-ppm[0]
def find_index(this_ppm):
    return int(np.ceil((this_ppm-ppm[0])/interval))

def cphase(angle):
    return np.exp(1j*np.radians(angle))

peaks = [(-57.86,-57.89), (-57.97,-58.3), (-58.40,-58.44), (-207.875,-207.92), (-207.94,-208.05), (-208.2375,-208.2775) ]
phases = [-65.0,             -65.0,            114.5,             180.0,                0.0,               0.0]

cphases = [ cphase(i) for i in phases ]

In [ ]:
spectrum_number = 1      
peak_number = 5          
spectrum = raw_spectra[spectrum_number]

plt.figure(figsize=(18,4))
peak = peaks[peak_number]
phase = cphases[peak_number]

plt.plot(ppm, np.real(spectrum*phase), "k")

center = np.average(peak)
x_axis_range = 0.5
start = center+x_axis_range
end = center-x_axis_range
plt.xlim(start,end)
plt.ylim(-1E6, 5E6)     

peak_start = find_index(peak[0])
peak_end = find_index(peak[1])
peak_x = ppm[peak_start:peak_end]
peak_y = np.real(spectrum[peak_start:peak_end]*phase)
plt.plot(peak_x,peak_y,"bo")

plt.xlabel("19F chemical shift (ppm)")
plt.xticks(np.linspace(end,start,41))

ax = plt.gca()
ax.tick_params(top="off")
ax.get_xaxis().set_tick_params(length=5,direction='out', width=1)
locs, labels = plt.xticks()
plt.setp(labels, rotation=90)

plt.show()

In [ ]:
def compute_baseline(spectrum, order=2, clip_below = -60000, clip_above = 60000, plot=False):
    noise_x = ppm.copy()
    noise_y = np.real(spectrum.copy())
    mask = np.ones(len(ppm), dtype=bool)
    for peak in peaks:
        index_low  = find_index(peak[0])
        index_high = find_index(peak[1])
        mask[index_low:index_high] = False
    noise_x = noise_x[mask]
    noise_y = noise_y[mask]
    noise_y = np.clip(noise_y, clip_below, clip_above)

    poly_coeff = np.polyfit(noise_x,noise_y,order)
    baseline_func = np.poly1d(poly_coeff)
    baseline = baseline_func(ppm)
    RMSE = np.sqrt(np.mean(np.square(noise_y-baseline_func(noise_x))))

    if plot:
        plt.figure(figsize=(18,4))
        plt.plot(ppm, np.real(spectrum), "k")
        plt.plot(ppm, baseline, "r")
        y_minus_limit = 1.67*clip_below if clip_below < 0.0 else 0.6*clip_below
        y_plus_limit = 1.67*clip_above if clip_above > 0.0 else 0.6*clip_above
        plt.ylim(y_minus_limit,y_plus_limit)
        plt.xlabel("19F chemical shift (ppm)")
        ax = plt.gca()
        ax.tick_params(top="off")
        ax.get_xaxis().set_tick_params(length=5,direction='out', width=1)
        locs, labels = plt.xticks()
        plt.setp(labels, rotation=90)
        plt.show()
    else:
        print "%.1E " % (RMSE / 1E5),
        for i in poly_coeff:
            print "%6.2f" % i,
        print
    return baseline

baselines = [ compute_baseline(spectrum) for spectrum in raw_spectra ]
subtracted_spectra = [ spectrum - baseline for spectrum, baseline in zip(raw_spectra,baselines) ]

In [ ]:
def compute_signal_to_noise(spectrum, noise=(-95.00, -100.00), plot=False):
    xy = np.array([ppm,spectrum])
    min_value2 = find_index(noise[0])
    max_value2 = find_index(noise[1])
    y_noise  = np.real(xy[1,min_value2:max_value2])
    zero_level   = np.mean(y_noise)

    signal_to_noise = []
    for i,peak in enumerate(peaks):
        min_value1 = find_index(peak[0])
        max_value1 = find_index(peak[1])
        y_signal = np.real(xy[1,min_value1:max_value1]*cphases[i])

        signal_level = np.max(y_signal - zero_level)/2.0
        noise_level  = np.sqrt(np.mean(np.square(y_noise-zero_level))) 
        signal_to_noise.append(signal_level / noise_level)
        
        if plot:
            print "%.2E %.2E" % (signal_level, noise_level)
            print zero_level
            plt.plot(x_signal,y_signal,"r")
            plt.plot(x_noise,y_noise,"b")
            #plt.ylim(-5E4,5E4)
            plt.show()

    return signal_to_noise

signal_to_noise_list = [ compute_signal_to_noise(spectrum) for spectrum in subtracted_spectra ]
signal_to_noise = DataFrame(signal_to_noise_list)
experiment_numbers = [ i+1 for i in range(len(raw_spectra)) ]
signal_to_noise["run"] = experiment_numbers
signal_to_noise.set_index("run",drop=True,inplace=True)
peak_numbers = range(len(peaks))
peak_numbers = ["peak %d" % (i+1) for i in peak_numbers]
signal_to_noise.columns = peak_numbers
#display(signal_to_noise)

avg_signal_to_noise = signal_to_noise.mean()
avg_signal_to_noise = avg_signal_to_noise.apply(lambda x : "%.0f" % x)
avg_signal_to_noise.name = "S/N"
display(avg_signal_to_noise)

In [ ]:
def integrate(spectrum, i, peak):
    index_low = find_index(peak[0])
    index_high = find_index(peak[1])
    return np.sum(np.real(spectrum[index_low:index_high]*cphases[i]))

In [ ]:
# integrate all spectra
results=[]
for spectrum in subtracted_spectra:
    integrals = []
    for i,peak in enumerate(peaks):
        integrals.append(integrate(spectrum,i,peak))
    integrals = np.array(integrals)
    integrals = integrals/2E7                          
    #integrals = 10.00 * integrals / integrals[3]      
    results.append(integrals)
    
integrations = DataFrame(results)

integrations.columns = peak_numbers
integrations["run"] = experiment_numbers
integrations.set_index("run",drop=True,inplace=True)

display(integrations)

In [ ]:
print "n = %d" % len(integrations)
mean = integrations.mean().apply(lambda x : "%.4f" % x)
stdev = integrations.std().apply(lambda x : "%.4f" % x)
cov = (100.0*integrations.std()/integrations.mean()).apply(lambda x : "%.2f%%" % x)
stderr = (integrations.std() / np.sqrt(len(integrations))).apply(lambda x : "%.4f" % x)
stderr_cov = (100*integrations.std()/(integrations.mean()*np.sqrt(len(integrations)))).apply(lambda x : "%.2f%%" % x)
headings = ["avg", "stdev", "cov", "stderr", "stderr_cov", "S/N"]
summary_df = DataFrame([mean, stdev, cov, stderr, stderr_cov, avg_signal_to_noise], index=headings)
display(summary_df)